In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# pip install pickle5

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import math

In [2]:
pip install pickle5

In [5]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False, preprocess=False):
    if city == "all" and split == "train":
        all_inputs = None
        all_outputs = None
        for city in cities:
            if all_inputs is None:
                f_in = ROOT_PATH + split + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                all_inputs = np.asarray(inputs).astype(np.float64)
            else:
                f_in = ROOT_PATH + split + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                inputs = np.asarray(inputs).astype(np.float64)
                all_inputs = np.concatenate([all_inputs, inputs], axis=0)

            if all_outputs is None:
                f_out = ROOT_PATH + split + "/" + city + "_outputs"
                outputs = pickle.load(open(f_out, "rb"))
                all_outputs = np.asarray(outputs).astype(np.float64)
            else:
                f_out = ROOT_PATH + split + "/" + city + "_outputs"
                outputs = pickle.load(open(f_out, "rb"))
                outputs = np.asarray(outputs).astype(np.float64)
                all_outputs = np.concatenate([all_outputs, outputs], axis=0)
            
        inputs = all_inputs
        outputs = all_outputs
        
    elif city == "all" and split=="test":
        all_test_inputs = None
        
        for city in cities:
            if all_test_inputs is None:
                f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                all_test_inputs = np.asarray(inputs).astype(np.float64)
            else:
                f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                inputs = np.asarray(inputs).astype(np.float64)
                all_test_inputs = np.concatenate([all_test_inputs, inputs], axis=0)

        outputs = None
        
        inputs = all_test_inputs
        

    if preprocess:
        preprocessed_inp = []
        preprocessed_out = []
        for i in range(inputs.shape[0]):
            temp_i, temp_o = preprocess_single(inputs[i], outputs[i])
            preprocessed_inp.append(temp_i)
            preprocessed_out.append(temp_o)

        preprocessed_inp = np.array(preprocessed_inp)
        preprocessed_out = np.array(preprocessed_out)
        
        return preprocessed_inp, preprocessed_out
    
    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None, normalize=False, preprocess=False):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=normalize, preprocess=preprocess)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

In [ ]:
inp, out = get_city_trajectories(city="all")

In [38]:
def plot_sample(num):
    plt.plot(inp[num][:, 0], inp[num][:, 1], label="input")
    plt.plot(out[num][:, 0], out[num][:, 1], label="output")
    plt.title("Visualization of sample-{} of the entire training dataset".format(num))
    plt.legend()
    plt.show()

In [40]:
plot_sample(20021)

In [ ]:
# here inp and out are single-target
def preprocess_single(inp, out):
    x_i = inp[0][0]
    y_i = inp[0][1]
    
    prep_inp = np.concatenate([(inp[:, 0] - x_i).reshape(50, 1), (inp[:, 1] - y_i).reshape(50, 1)], axis=1)
    
    if out is None:
        return prep_inp, None

    prep_out = np.concatenate([(out[:, 0] - x_i).reshape(60, 1), (out[:, 1] - y_i).reshape(60, 1)], axis=1)
    
    return prep_inp, prep_out

In [ ]:
device = torch.device("cuda:0")
device

In [ ]:
class Pred(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.encoder1 = nn.Sequential(
            nn.Linear(862, 512),
            nn.ReLU()
#             nn.Dropout(0.5)
        )
        
        self.encoder2 = nn.Sequential(
            nn.Linear(862, 512),
            nn.ReLU()
#             nn.Dropout(0.5)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(862, 512),
            nn.ReLU(),
            nn.Linear(862, 512)
        )
        
        self.layer_norm = nn.LayerNorm(512)
        
        
    def forward(self, x):
        
        x = x.to(device)
        
        x = x.reshape(-1, 350).float()
        y = torch.zeros(len(x), 512)
        
        y = y.to(device)
        
        y = self.encoder1(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        y = self.encoder2(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        
        y = self.encoder1(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        y = self.encoder2(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        
        y = self.encoder1(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        y = self.encoder2(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        
        y = self.encoder1(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        y = self.encoder2(torch.cat((x, y), 1))
        y = self.layer_norm(y)
        
        output = self.decoder(torch.cat((x, y), 1))
        
        output = output.reshape(-1, 60, 2)
        
        return output

In [ ]:
pred = Pred().to(device)
opt = torch.optim.Adam(pred.parameters(), lr=0.0005)

In [ ]:
# batch_sz = 512  # batch size 
# city = 'palo-alto' 
# split = 'train'
# train_dataset  = ArgoverseDataset(city = city, split = split, normalize=False, preprocess=True)
# train_loader = DataLoader(train_dataset, batch_size=batch_sz)

In [ ]:
batch_sz = 512  # batch size 
city = "all" 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split, normalize=False, preprocess=True)

train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train, valid = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

train_loader = DataLoader(train, batch_size=batch_sz)
valid_loader = DataLoader(valid, batch_size=batch_sz)

In [ ]:
i, o = get_city_trajectories(city="all", split="train", normalized=False, preprocess=True)

In [ ]:
# p_i, p_o = get_city_trajectories(city="palo-alto", split="train", normalized=False)
# train_combined = np.concatenate([p_i, p_o], axis=1)
# global_x_mean = np.mean(train_combined[:, :, 0].reshape(-1))
# global_x_std = np.std(train_combined[:, :, 0].reshape(-1))
# global_y_mean = np.mean(train_combined[:, :, 1].reshape(-1))
# global_y_std = np.std(train_combined[:, :, 1].reshape(-1))

In [ ]:
# global_x_mean

In [ ]:
def find_velocity(xy):
    x = xy[:,:,0]
    y = xy[:,:,1]
    diff_x = torch.diff(x)/0.1
    diff_y = torch.diff(y)/0.1
    diff_x = torch.cat((diff_x, diff_x[:,-1:]),1).reshape(-1,50,1)
    diff_y = torch.cat((diff_y, diff_y[:,-1:]),1).reshape(-1,50,1)
    return torch.cat((x.reshape(-1,50,1),y.reshape(-1,50,1),diff_x,diff_y), 2)

In [ ]:
def find_accel(xy):
    x = xy[:,:,2]
    y = xy[:,:,3]
    diff_x = torch.diff(x)/0.1
    diff_y = torch.diff(y)/0.1
    diff_x = torch.cat((diff_x, diff_x[:,-1:]),1).reshape(-1,50,1)
    diff_y = torch.cat((diff_y, diff_y[:,-1:]),1).reshape(-1,50,1)
    return torch.cat((xy,diff_x,diff_y), 2)

In [ ]:
def find_angular_velocity(xy):
    x_i, x_f = xy[:,0,0], xy[:,-1,0]
    y_i, y_f = xy[:,0,1], xy[:,-1,1]

    radius = (((x_f - x_i)**2 + (y_f - y_i)**2)**0.5).reshape(len(xy), 1)   # shape of 512 x 1
    
#     broad_cast_radius = torch.tensor([[r] * 50 for r in radius]).to(device)  # shape of 512 x 50, the radius for each 50 timstamps is the same
    broad_cast_radius = radius.expand(len(xy), 50).to(device)
    
    vx = xy[:,:,2]
    vy = xy[:,:,3]
    
    vr = vx + vy
    
    w = (vr / broad_cast_radius).reshape(-1, 50, 1).to(device)
    
    return torch.cat((xy, w), 2)

In [ ]:
def find_velocity_single(xy):
    x = xy[:,0]
    y = xy[:,1]
    diff_x = torch.diff(x)/0.1
    diff_y = torch.diff(y)/0.1
    diff_x = torch.cat((diff_x, diff_x[-1:]),0).reshape(50,1)
    diff_y = torch.cat((diff_y, diff_y[-1:]),0).reshape(50,1)
    return torch.cat((x.reshape(50,1),y.reshape(50,1),diff_x,diff_y), 1)

In [ ]:
def find_accel_single(xy):
    x = xy[:,2]
    y = xy[:,3]
    diff_x = torch.diff(x)/0.1
    diff_y = torch.diff(y)/0.1
    diff_x = torch.cat((diff_x, diff_x[-1:]),0).reshape(50,1)
    diff_y = torch.cat((diff_y, diff_y[-1:]),0).reshape(50,1)
    return torch.cat((xy, diff_x, diff_y), 1)

In [ ]:
def find_angular_single(xy):
    x = xy[:,0]
    y = xy[:,1]
    
    x_i, x_f = x[0], x[-1]
    y_i, y_f = y[0], y[-1]
    
    radius = (((x_f - x_i)**2 + (y_f - y_i)**2)**0.5)
    
    vx = xy[:,2]
    vy = xy[:,3]
    
    vr = (vx + vy)
    
    w = (vr / radius).reshape(50, 1)
    
    return torch.cat((xy, w), 1)

In [ ]:
def train(pred):
    loss_lst = []
    valid_loss_lst = []
    epochs = []
    for epoch in range(20):
        epochs.append(epoch)
        total_loss = 0
        i = 0
        for i_batch, sample_batch in enumerate(train_loader):
            inp, out = sample_batch

#             print(inp.shape)
#             print(inp)
            inp = inp.to(device)
            out = out.to(device)
            inp = find_velocity(inp)
            inp = find_accel(inp)
            inp = find_angular_velocity(inp)
        
            preds = pred(inp)
#             print(preds.shape)
#             print(out.shape)

            loss = ((preds - out) ** 2).sum().to(device)

            opt.zero_grad()
            loss.backward()
            opt.step()
            
            i += len(inp) * 60 * 2
            
            total_loss += loss.item()
            
        loss_lst.append(total_loss / i)
        if epoch >= 0:
            print('epoch {} train loss: {}'.format(epoch, total_loss / i))
        
        
            # calculate validation loss
            total_valid_loss = 0
            i = 0
            for i_batch, sample_batch in enumerate(valid_loader):
                inp, out = sample_batch
                
                inp = inp.to(device)
                out = out.to(device)
                
                inp = find_velocity(inp)
                inp = find_accel(inp)
                inp = find_angular_velocity(inp)
                
        #             print(inp.shape)
        #             print(inp)
                preds_valid = pred(inp)
        #             print(preds.shape)
        #             print(out.shape)
#                 inp = inp.to(device)
#                 out = out.to(device)
                valid_loss = ((preds_valid - out) ** 2).sum().to(device)

                i += len(inp) * 60 * 2

                total_valid_loss += loss.item()

            valid_loss_lst.append(total_valid_loss / i)

            print('epoch {} valid loss: {}'.format(epoch, total_valid_loss / i))
            print('--------------------------------------------------------------')
            print('\n')
                  
    plt.plot(epochs, loss_lst, label="train_loss over time")
    plt.plot(epochs, valid_loss_lst, label="valid_loss over time")
    plt.legend()
    
    return epochs, loss_lst, valid_loss_lst

In [ ]:
import time

In [ ]:
start = time.time()
print("Starts training process")
epochs, loss_lst, valid_loss_lst = train(pred)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [ ]:
plt.plot(epochs, loss_lst, label="train_loss over time")
plt.plot(epochs, valid_loss_lst, label="valid_loss over time")
plt.xlabel("number of epochs")
plt.ylabel("MSE Loss")
plt.xticks(epochs)
plt.legend()

In [ ]:
p_test_i, p_test_o = get_city_trajectories(city="all", split="test")

In [ ]:
p_test_i.shape

In [ ]:
# def normalize(p_test_i):
#     return np.concatenate([((p_test_i[:, 0] - global_x_mean)/global_x_std).reshape(1,50,1), \
#                 ((p_test_i[:, 1] - global_y_mean)/global_y_std).reshape(1,50,1)], axis=2)

In [ ]:
def predict(model, test_inp):
    
    x_i = test_inp[0][0]
    y_i = test_inp[0][1]
    
    test_inp = preprocess_single(test_inp, None)[0]
    
    test_inp = torch.tensor(test_inp).to(device)
    test_inp = find_velocity_single(test_inp)
    test_inp = find_accel_single(test_inp)
    test_inp = find_angular_single(test_inp)
    
    pred = model(test_inp)[0]
        
    pred_x = pred[:, 0] + x_i
    pred_y = pred[:, 1] + y_i
    
    return pred_x, pred_y

In [ ]:
# here inp and out are single-target
def preprocess_single(inp, out):
    x_i = inp[0][0]
    y_i = inp[0][1]
    
    prep_inp = np.concatenate([(inp[:, 0] - x_i).reshape(50, 1), (inp[:, 1] - y_i).reshape(50, 1)], axis=1)
    
    if out is None:
        return prep_inp, None

    prep_out = np.concatenate([(out[:, 0] - x_i).reshape(60, 1), (out[:, 1] - y_i).reshape(60, 1)], axis=1)
    
    return prep_inp, prep_out

In [ ]:
palo_df = pd.DataFrame()
for i in range(len(p_test_i)):
    
#     predict = model_3(torch.tensor(normalize(p_test_i[i])))
    p_x, p_y = predict(model, p_test_i[i])
    
    if i % 100 == 0:
        print("predicts for " + str(i) + " is done")
    temp_dict = {}
    for j in range(0, 120, 2):
        col_x = 'v' + str(j)
        col_y = 'v' + str(j+1)
        row = j // 2
#         temp_dict[col_x] = predict[0][row][0].item() * global_x_std + global_x_mean
#         temp_dict[col_y] = predict[0][row][1].item() * global_y_std + global_y_mean
        temp_dict[col_x] = p_x[row].item()
        temp_dict[col_y] = p_y[row].item()
    
    temp_idx_name = str(i) + "_" + "palo-alto"
    temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
    palo_df = pd.concat([palo_df, temp_final])

In [ ]:
palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model9.csv", index=False)

In [ ]:
torch.save(model.state_dict(), "model9.pt")

In [ ]:
# test_model = Pred()
# test_model.load_state_dict(torch.load("./mlp_with_preprocess_palo_alto.pt"))